In [5]:
%load_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.hub
import random

import torchvision
import torchvision.utils
from torchvision.models import detection
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.ssd import SSD

import dataset as data
import utils as utils
import pickle
import attackMethods as am

print(torch.__version__)
print(torch.cuda_version)
print(torchvision.__version__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1.10.0+cu113
11.3
0.11.1+cu113


In [4]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

dataset = data.AdversarialDataset((640,640)) # todo: use resize to pull picture in batch

loss = am.lossObjectness


batch_size = 1

train_loader  = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=False)

device = torch.device("cpu")
#device = torch.device("cuda:0")
#'cuda:0' or 'cpu'

In [3]:
#train on pictures from dset (pictures with persons in personIndex list)

model.eval()
model = model.float().to(device)

for param in model.parameters():
    param.requires_grad = False

patch = am.generatePatch()
#patch = am.patchFromImage("meme.jpg")
patch = patch.to(device)
patch.requires_grad = True

TVCoeff = 0.000000001
GradRate = 0.03

augmentations = torchvision.transforms.Compose([
        torchvision.transforms.ColorJitter(brightness=0.4, contrast=0.2, saturation=0.2, hue=0.05),
        torchvision.transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 0.3)),
        torchvision.transforms.RandomPerspective(distortion_scale=0.2, p=1.0),
        torchvision.transforms.RandomRotation(degrees=(-30, 30)),])

#personIndex = []

epoches = 1

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

for epoch in range(epoches):
    imageCounter = 0
    for image, label in train_loader:
        imageCounter += batch_size
        for k in range(1):
            cv2.imshow("patch", cv2.cvtColor(cv2.resize(data.TenToIm(patch.clone().detach().to(torch.device("cpu"))/255), (300,300)), cv2.COLOR_RGB2BGR))

            attackedImage = []

            torch.cuda.empty_cache()

            for im in image:
                attackedImage.append(data.ImToTen(im).to(device))

            #for attackedIm in attackedImage:
            #    attackedIm.requires_grad = True

            with torch.no_grad():
                clearPredict = model(attackedImage)

            augmented_patch = augmentations(patch)

            for i in range(len(attackedImage)):
                for l in label[i]:
                    attackedImage[i] = am.setPatch(attackedImage[i], augmented_patch, l, 0.2, device) 

            predict = model(attackedImage)

            costs = []

            for i in range(len(clearPredict)):
                cost = loss(clearPredict[i], predict[i]) + TVCoeff * am.TV(patch, device)
                if cost < 0.1:
                    continue
                try:
                    grad = torch.autograd.grad(cost, patch, retain_graph=False, create_graph=False,  allow_unused=True)[0]
                    if grad != None:
                        patch = patch - GradRate*grad.sign()
                except:
                    pass
                costs.append(cost.detach().cpu())
            

            print("ep:", epoch,"epoch_progress:", imageCounter/len(dataset), "loss:", np.mean(np.asarray(costs)))

            for attackedIm in attackedImage:
                attackedIm.detach()

            key = cv2.waitKey(1)
            if key & 0xFF == ord('q'):
                cv2.destroyAllWindows() 
                utils.SavePatch(patch.cpu(), "patch - meme")
                raise StopExecution


c:\Users\Степан\Desktop\учеба\Диплом\FromUbuntu\diplomfile\dataset.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return (torch.tensor(image).permute(2, 0, 1).float() / 255)


TypeError: transpose() received an invalid combination of arguments - got (tuple), but expected one of:
 * (int dim0, int dim1)
 * (name dim0, name dim1)


In [6]:
torch.cuda.empty_cache()
cv2.destroyAllWindows() 
utils.ShowPatch(patch.cpu())

AttributeError: module 'utils' has no attribute 'ShowPatch'

In [8]:
#image, label = dataset[personIndex[7]]
#network parameters requar. grad

In [9]:
patch = utils.LoadPatch("patch - meme")

In [8]:
utils.SavePatch(patch.cpu(), "patch - meme")

In [24]:
with open('0_4784.pickle', 'wb') as f:
    pickle.dump(personIndex, f)

In [4]:
with open('0_4784.pickle', 'rb') as f:
    personIndex = pickle.load(f)

In [68]:
image, label = dataset[personIndex[7]]
patch = utils.TrainOneImage(model, loss, image, label, epoches=10)

ep: 0  loss: tensor(1.1819, grad_fn=<AddBackward0>)
ep: 1  loss: tensor(1.1061, grad_fn=<AddBackward0>)
ep: 2  loss: tensor(1.0346, grad_fn=<AddBackward0>)
ep: 3  loss: tensor(0.9632, grad_fn=<AddBackward0>)
ep: 4  loss: tensor(0.8850, grad_fn=<AddBackward0>)
ep: 5  loss: tensor(0.8303, grad_fn=<AddBackward0>)
ep: 6  loss: tensor(0.7588, grad_fn=<AddBackward0>)
ep: 7  loss: tensor(0.6875, grad_fn=<AddBackward0>)
ep: 8  loss: tensor(0.6162, grad_fn=<AddBackward0>)
ep: 9  loss: tensor(0.5449, grad_fn=<AddBackward0>)


In [23]:
for i in range(20):
    image, label = dataset[personIndex[i]]
    utils.CheckPatch(patch, image, label, model, device, loss)

Clear loss: tensor(1.1095, grad_fn=<MulBackward0>) Attacked loss: tensor(1.1095, grad_fn=<MulBackward0>)
Clear loss: tensor(1.2138, grad_fn=<MulBackward0>) Attacked loss: tensor(1.1075, grad_fn=<MulBackward0>)
Clear loss: tensor(1.1265, grad_fn=<MulBackward0>) Attacked loss: tensor(1.1043, grad_fn=<MulBackward0>)
Clear loss: tensor(1.1616, grad_fn=<MulBackward0>) Attacked loss: tensor(1.2186, grad_fn=<MulBackward0>)
Clear loss: tensor(1.1064, grad_fn=<MulBackward0>) Attacked loss: tensor(1.1702, grad_fn=<MulBackward0>)
Clear loss: tensor(1.8352, grad_fn=<MulBackward0>) Attacked loss: tensor(1.1531, grad_fn=<MulBackward0>)
Clear loss: tensor(1.4608, grad_fn=<MulBackward0>) Attacked loss: tensor(1.1437, grad_fn=<MulBackward0>)
Clear loss: tensor(1.1680, grad_fn=<MulBackward0>) Attacked loss: tensor(1.1739, grad_fn=<MulBackward0>)
Clear loss: tensor(1.1299, grad_fn=<MulBackward0>) Attacked loss: tensor(1.1785, grad_fn=<MulBackward0>)
Clear loss: tensor(1.6535, grad_fn=<MulBackward0>) Atta